In [63]:
import pandas as pd
from geopandas import GeoDataFrame
import geopandas as gpd
from shapely.geometry import Point,Polygon
from geopy.geocoders import Nominatim
from sodapy import Socrata
import json
import datetime
import os
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
%reload_ext autoreload
%autoreload 2

# load api_keys
with open("api_keys_kat.json") as f:
    keys = json.load(f)

with open("apikeys.json") as f:
    planet_keys = json.load(f)

PLANET_API_KEY = planet_keys["PLANET_API_KEY"]

In [64]:
LAX = keys["LAX"]

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata(LAX["url"], 
                 LAX["key"],
                 username=LAX["username"], 
                 password=LAX["password"])


In [65]:
start_date = datetime.datetime(2017,1,1)
end_date = datetime.datetime(2017,12,31)
cost = 1000000
query = "SELECT * WHERE issue_date > '{0}' AND issue_date < '{1}' AND valuation > {2}"
query = query.format(start_date.strftime("%Y-%m-%d"),end_date.strftime("%Y-%m-%d"),cost)
print(query)
results = client.get(LAX["resource"],query=query )
results_df = pd.DataFrame.from_records(results)

SELECT * WHERE issue_date > '2017-01-01' AND issue_date < '2017-12-31' AND valuation > 1000000


In [66]:
print(len(results_df))
print(results_df.keys())
print(results_df["location_1"])
# DROP ROWS THAT DON'T HAVE LAT OR LONG DATA
results_df = results_df.dropna(subset=["location_1"])
results_df

854
Index([u'address_end', u'address_fraction_end', u'address_fraction_start',
       u'address_start', u'applicant_address_1', u'applicant_address_2',
       u'applicant_address_3', u'applicant_business_name',
       u'applicant_first_name', u'applicant_last_name',
       u'applicant_relationship', u'assessor_book', u'assessor_page',
       u'assessor_parcel', u'block', u'census_tract', u'contractor_address',
       u'contractor_city', u'contractor_state', u'contractors_business_name',
       u'council_district', u'floor_area_l_a_building_code_definition',
       u'floor_area_l_a_zoning_code_definition', u'initiating_office',
       u'issue_date', u'latest_status', u'license', u'license_expiration_date',
       u'license_type', u'location_1', u'lot', u'occupancy',
       u'of_residential_dwelling_units', u'of_stories', u'pcis_permit',
       u'permit_category', u'permit_sub_type', u'permit_type',
       u'principal_first_name', u'principal_last_name',
       u'principal_middle_name', 

,address_end,address_fraction_end,address_fraction_start,address_start,applicant_address_1,applicant_address_2,applicant_address_3,applicant_business_name,applicant_first_name,applicant_last_name,...,street_name,street_suffix,suffix_direction,tract,unit_range_end,unit_range_start,valuation,work_description,zip_code,zone
1,700,NaN,NaN,700,1715 W. 135TH,NaN,"GARDDENA, CA",NaN,JOSE,GUDIEL,...,WORLD,WAY,NaN,RANCHO SAUSAL REDONDO,NaN,NaN,1379815,SUPPLEMENTAL PERMIT TO 15016-10000-08908. DE...,90045,LAX
2,12828,NaN,NaN,12828,23660 STRATHERN,NaN,"WEST HILLS,CA",NaN,DOMINIC,HONG,...,RIVERSIDE,DR,NaN,TR 2590,NaN,NaN,10500000,(N) HOTEL W/ SUBTERANEAN GARAGE BELOW,91607,(T)(Q)RAS4-1VL-RIO
3,1600,NaN,NaN,1600,NaN,NaN,NaN,NaN,OCCIDENTAL,NaN,...,CAMPUS,ROAD,NaN,TR 607,NaN,NaN,2000000,FOUNDATION EARLY START PERMIT ONLY. THIS APPLI...,90032,R1-1
4,11050,NaN,NaN,11050,1453 CLUB VIEW DR,NaN,"LA,CA",NaN,FRANK,AFARI,...,HARTSOOK,ST,NaN,HARTSOOK TRACT,48,1,6008535,"NEW 3-STORY TYPE V-A, 48-UNIT APARTMENT BUILDI...",91601,R4-1VL
5,11000,NaN,NaN,11000,1636 WESTWOOD BL,#201,"L.A., CA",NaN,JASON,MASSABAND,...,CHALON,ROAD,NaN,BEL-AIR,NaN,NaN,1600000,New irregular shape 2- story over 1- level of ...,90077,RE20-1-H
6,1522,NaN,NaN,1522,3456 GOLDEN POPPY WAY,NaN,"YORBA LINDA, CA",NaN,SHEHZAD,KHALIQ,...,LOMITA,BLVD,NaN,TR 1589,NaN,1-14,2000000,NEW 14 UNIT APARTMENT BUILDING,90710,[Q]R3-1VL
7,5135,NaN,NaN,5135,3834 WILLAT AVE,NaN,"CULVER CITY, CA",NaN,STD,LEMONS,...,ADAMS,BLVD,NaN,TR 1566,10,1,1925000,"PROPOSED 3-STORY, 10-UNIT MIXED USE BUILDING W...",90016,C2-1
8,5101,NaN,NaN,5101,232 N LAKE AVE,STE 217,"PASADENA, CA",JAYCOCAL ENGINEERING,MARIA,CHEN,...,LANKERSHIM,BLVD,NaN,TR 2170,NaN,NaN,40000000,INSTALLATION OF NEW PLUMBING SYSTEM FOR (2) MI...,91601,C4-1-CA
9,11051,NaN,NaN,11051,NaN,NaN,NaN,NaN,MARCUS,PARIS,...,PENDLETON,ST,NaN,LOS ANGELES LAND AND WATER CO'S SUBDIVISION OF...,NaN,NaN,1200000,TI to existing Warehouse to build out accessor...,91352,[Q]M2-1-G-CUGU
12,6801,NaN,NaN,6801,NaN,NaN,NaN,NaN,ALEX,NaN,...,HOLLYWOOD,BLVD,NaN,TR 52496,NaN,C5-525,1800000,TI at restaurant on the 5th floor. ENCLOS...,90028,C4-2D-SN


In [69]:

def point_to_box(lon,lat,sz=[350,350]):
    # stack overflow copy pasta on lat/lon to meters.
    m_per_deg_lat = 111132.954 - 559.822 * math.cos( 2 * lat) + 1.175 * math.cos( 4 * lat)
    m_per_deg_lon = 111132.954 * math.cos( lat )
    m_per_deg_lat_1 = 1.0 / m_per_deg_lat
    m_per_deg_lon_1 = 1.0 / m_per_deg_lon
    dx = (sz[0]/2.0)*m_per_deg_lon_1
    dy = (sz[1]/2.0)*m_per_deg_lat_1
    geom = {"geometry": {
            "type": "Polygon",
            "coordinates": []  }}   
    dummy = [[-1.0,-1.0],[-1.0,1.0],[1.0,1.0],[1.0,-1.0],[-1.0,-1.0]]
    corners = [[(d[0]*dx)+lon,(d[1]*dy)+lat] for d in dummy]
    geom["geometry"]["coordinates"].append(corners)
    return geom
  
polys = []
shapes = [] 
print 
for point in results_df["location_1"].tolist():
    p = point["coordinates"]
    box = point_to_box(p[0],p[1])
    polys.append(box)
    s = Polygon(shell=box["geometry"]["coordinates"][0])    
    shapes.append(s)
results_df.insert(loc=0, column='AOI', value=polys)
results_df.insert(loc=0, column='shapes', value=polys)

results_df.head()
len(results_df)

ValueError: cannot insert AOI, already exists

In [70]:
# Basemap Mosaic (v1 API)
mosaicsSeries = 'global_quarterly_2017q1_mosaic'
# Planet tile server base URL (Planet Explorer Mosaics Tiles)
mosaicsTilesURL_base = 'https://tiles0.planet.com/experimental/mosaics/planet-tiles/' + mosaicsSeries + '/gmap/{z}/{x}/{y}.png'
# Planet tile server url
mosaicsTilesURL = mosaicsTilesURL_base + '?api_key=' + PLANET_API_KEY
# Map Settings 
# Define colors
colors = {'blue': "#009da5"}
# Define initial map center lat/long
center = [34.0522, -118.2437]
# Define initial map zoom level
zoom = 9
# Set Map Tiles URL
planetMapTiles = TileLayer(url= mosaicsTilesURL)
# Create the map
m = Map(
    center=center, 
    zoom=zoom,
    default_tiles = planetMapTiles # Uncomment to use Planet.com basemap
)
# Define the draw tool type options
polygon = {'shapeOptions': {'color': colors['blue']}}
rectangle = {'shapeOptions': {'color': colors['blue']}} 

# Create the draw controls
# @see https://github.com/ellisonbg/ipyleaflet/blob/master/ipyleaflet/leaflet.py#L293
dc = DrawControl(
    polygon = polygon,
    rectangle = rectangle
)
# Initialize an action counter variable
actionCount = 0
AOIs = {}

# Register the draw controls handler
def handle_draw(self, action, geo_json):
    # Increment the action counter
    global actionCount
    actionCount += 1
    # Remove the `style` property from the GeoJSON
    geo_json['properties'] = {}
    # Convert geo_json output to a string and prettify (indent & replace ' with ")
    geojsonStr = json.dumps(geo_json, indent=2).replace("'", '"')
    AOIs[actionCount] = json.loads(geojsonStr)
    

    
    
# Attach the draw handler to the draw controls `on_draw` event
dc.on_draw(handle_draw)
m.add_control(dc)
m

for p in polys:
    feat = {'geometry':p["geometry"],"properties":{
            'style':{'color': "#00FF00",'fillColor': "#00FF00",'fillOpacity': 0.5,'weight': 1}},
            'type':u"Feature"}
    # convert to geojson
    gjson = GeoJSON(data=feat)
    # add it our map
    m.add_layer(gjson)
m